# 2. 팩터 분석

제조업 재무지표·수익률 데이터를 전처리한 뒤, (원) 단위 컬럼을 제거하고 PCA·요인분석을 수행합니다.  
`data/` 폴더에 `제조업_수익률.csv`가 있어야 합니다 (01 데이터 전처리 또는 별도 공시 데이터 병합 결과).

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))
DATA_DIR = PROJECT_ROOT / "data"

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 결측 제거 (금융 기업 등)
df = pd.read_csv(DATA_DIR / '제조업_수익률.csv', dtype={'주식코드': str}).dropna()

# (원) 단위 컬럼 제거
cols_drop = [c for c in df.columns if '원)' in c]
df.drop(columns=cols_drop, inplace=True)

# 수익률 컬럼명 통일 (수익율 -> 수익률)
if '수익율' in df.columns:
    df.rename(columns={'수익율': '수익률'}, inplace=True)
df.head()

In [ ]:
# 수익률 제외한 재무지표만 사용
feature_cols = [c for c in df.columns if c != '수익률']
X = df[feature_cols].copy()
X = StandardScaler().fit_transform(X)

# PCA
pca = PCA(n_components=min(10, X.shape[1], X.shape[0]))
pca.fit(X)
print('설명 분산 비율:', pca.explained_variance_ratio_)

In [ ]:
# 요인분석 (예: 5개 요인)
n_factors = 5
X_fa = df[feature_cols].dropna(axis=1, how='all')
fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax')
fa.fit(X_fa)
loadings = pd.DataFrame(
    fa.loadings_,
    index=X_fa.columns,
    columns=[f'Factor{i+1}' for i in range(n_factors)]
)
loadings.round(3)